In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Data preprocessing

In [4]:
df=pd.read_csv('movies.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
new_df=df.copy()

In [7]:
new_df.shape

(4803, 24)

In [8]:
selected_features=['genres','keywords','tagline','cast','director']

In [9]:
new_df.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [10]:
new_df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [11]:
for feature in selected_features:
  new_df[feature]=new_df[feature].fillna('')

In [13]:
combined_features=new_df['genres']+' '+new_df['keywords']+' '+new_df['tagline']+' '
+new_df['cast']+' '+new_df['director']

combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801                            A New Yorker in Shanghai 
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [18]:
# converting the text data to feature vectors
vectorizer=TfidfVectorizer()
feature_vectors=vectorizer.fit_transform(combined_features)

In [20]:
print(feature_vectors)

  (0, 4498)	0.35977146596126014
  (0, 4348)	0.11969800165259886
  (0, 7005)	0.16951497952634928
  (0, 6257)	0.09428706610088097
  (0, 2102)	0.3176552747616365
  (0, 5741)	0.2828344413384406
  (0, 1321)	0.33000816200564936
  (0, 6802)	0.16592099630102675
  (0, 5793)	0.44903494633034546
  (0, 2576)	0.21772343390141816
  (0, 1244)	0.2968078745280208
  (0, 1585)	0.2828344413384406
  (0, 2370)	0.13702838223474842
  (0, 5427)	0.13702838223474842
  (0, 2296)	0.14687069071321432
  (0, 190)	0.11927285589220428
  (0, 155)	0.10392045738902624
  (1, 646)	0.2256283604141521
  (1, 2078)	0.2363184185838419
  (1, 455)	0.1983798683793789
  (1, 3637)	0.15190171604782024
  (1, 4374)	0.14471052044386193
  (1, 3724)	0.14052089954370584
  (1, 1361)	0.26118368008546655
  (1, 6401)	0.3180638946881614
  :	:
  (4799, 4348)	0.11911590179532587
  (4799, 6257)	0.0938285414224214
  (4800, 5621)	0.3545557766157617
  (4800, 6187)	0.3488844346046621
  (4800, 4193)	0.37207260176999274
  (4800, 2416)	0.27494012230371234

##Cosine similarity

In [21]:
#getting the similarity scores
similarity=cosine_similarity(feature_vectors)

In [22]:
similarity

array([[1.        , 0.13264148, 0.02918063, ..., 0.        , 0.        ,
        0.        ],
       [0.13264148, 1.        , 0.06857325, ..., 0.08092515, 0.        ,
        0.        ],
       [0.02918063, 0.06857325, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.08092515, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [23]:
similarity.shape

(4803, 4803)

In [26]:
movie_name=input(' Enter your favourite movie name: ')

 Enter your favourite movie name: iron man


In [ ]:
titles=new_df['title'].to_list()
titles

In [27]:
#getting the most related movies to the user input
close_match=difflib.get_close_matches(movie_name,titles)
close_match

['Iron Man', 'Iron Man 3', 'Iron Man 2']

In [28]:
close_match=close_match[0]
close_match

'Iron Man'

In [29]:
# finding the index of the movie with the title

movie_index=new_df[new_df.title==close_match]['index'].values[0]
movie_index

68

In [ ]:
# getting a list of similarity_score 

similarity_score = list(enumerate(similarity[movie_index]))

In [34]:
sorted_similar_movies=sorted(similarity_score, key=lambda x:x[1],reverse=True)
sorted_similar_movies

[(68, 1.0),
 (182, 0.36842236847850435),
 (85, 0.36177525637338404),
 (79, 0.35158895830341075),
 (7, 0.31417483074321595),
 (511, 0.30984381296198027),
 (31, 0.28907571312750774),
 (16, 0.2832391971616202),
 (26, 0.2762310178462122),
 (169, 0.26858768677502487),
 (64, 0.26747930719007884),
 (101, 0.25202678165131964),
 (203, 0.2507152914517417),
 (174, 0.25008540042933247),
 (94, 0.24796552531528537),
 (788, 0.24357656021424162),
 (33, 0.2427288773643455),
 (4395, 0.24262873708697216),
 (46, 0.24068711633273707),
 (1740, 0.24026600416195137),
 (126, 0.22066891743860345),
 (2442, 0.21956307374198236),
 (122, 0.2121312352527577),
 (38, 0.21009100827731797),
 (3089, 0.2072349082516121),
 (242, 0.2034770342820388),
 (4450, 0.19512210152237008),
 (3385, 0.19282678174015802),
 (4401, 0.1927473804808672),
 (1956, 0.19048048259650122),
 (3386, 0.1900868185312059),
 (3993, 0.18672785803056405),
 (1451, 0.1831573906722762),
 (2900, 0.18263190740919277),
 (674, 0.18102355682460292),
 (4399, 0.18

In [36]:
# transforming values to titles
print('Movies suggested for you: \n')

i=1
for movie in sorted_similar_movies:
  index=movie[0]
  title=new_df[new_df.index==index]['title'].values[0]
  if(i<31):
    print(i,'.',title)
    i+=1

Movies suggested for you: 

1 . Iron Man
2 . Ant-Man
3 . Captain America: The Winter Soldier
4 . Iron Man 2
5 . Avengers: Age of Ultron
6 . X-Men
7 . Iron Man 3
8 . The Avengers
9 . Captain America: Civil War
10 . Captain America: The First Avenger
11 . X-Men: Apocalypse
12 . X-Men: First Class
13 . X2
14 . The Incredible Hulk
15 . Guardians of the Galaxy
16 . Deadpool
17 . X-Men: The Last Stand
18 . The Specials
19 . X-Men: Days of Future Past
20 . Kick-Ass 2
21 . Thor: The Dark World
22 . Southland Tales
23 . X-Men Origins: Wolverine
24 . The Amazing Spider-Man 2
25 . Krrish
26 . Fantastic Four
27 . American Hero
28 . March or Die
29 . The Helix... Loaded
30 . Red Sky


#Recommendation system

In [38]:
movie_name=input(' Enter your favourite movie name: ')

titles=new_df['title'].to_list()

close_match=difflib.get_close_matches(movie_name,titles)

close_match=close_match[0]

movie_index=new_df[new_df.title==close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[movie_index]))

sorted_similar_movies=sorted(similarity_score, key=lambda x:x[1],reverse=True)

print('Movies suggested for you: \n')

i=1
for movie in sorted_similar_movies:
  index=movie[0]
  title=new_df[new_df.index==index]['title'].values[0]
  if(i<21):
    print(i,'.',title)
    i+=1





 Enter your favourite movie name: bat man
Movies suggested for you: 

1 . Batman
2 . Batman & Robin
3 . Batman Returns
4 . The Dark Knight Rises
5 . Batman Begins
6 . The Dark Knight
7 . Superman
8 . A History of Violence
9 . Superman Returns
10 . Spider-Man 3
11 . Suicide Squad
12 . Bedazzled
13 . Spider-Man 2
14 . Salton Sea
15 . The One
16 . The New Guy
17 . Man of Steel
18 . Kick-Ass 2
19 . Superman II
20 . Catwoman
